In [2]:
from typing_extensions import TypedDict
from typing import List, Dict, Any, Union,Annotated,Sequence
from langchain.tools import tool
from langchain.chat_models import ChatOpenAI
from langgraph.graph import StateGraph, START, END, MessageGraph
from langchain_core.messages import HumanMessage, SystemMessage,BaseMessage
# from langgraph.prebuilt import ToolExecutor
from langgraph.prebuilt import create_react_agent, ToolNode
from langchain.tools import tool
from langchain_core.utils.function_calling import convert_to_openai_function,format_tool_to_openai_function
import operator
import os
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['OPENAI_URL'] = os.getenv('OPENAI_BASE_URL')
from langchain.chat_models import ChatOpenAI

In [3]:
class AgentState(TypedDict):
  messages: Annotated[Sequence[BaseMessage], operator.add] 

workflow = StateGraph(AgentState)

# # 节点使用 “状态”
# messages = state['messages']


# # 节点返回 “状态”
# return {"messages": [response]}

In [4]:
@tool
def multiply(first_number: int, second_number: int):
    """将2个数相乘"""
    return first_number * second_number

tools = [multiply]

tool_executor = ToolNode(tools)

In [5]:
model = ChatOpenAI(
    model="glm-4-flash",
)

functions = [format_tool_to_openai_function(t) for t in tools]
model = model.bind_functions(functions)

C:\Users\33282\AppData\Local\Temp\ipykernel_8400\1749589482.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  model = ChatOpenAI(
C:\Users\33282\AppData\Local\Temp\ipykernel_8400\1749589482.py:5: LangChainDeprecationWarning: The function `_format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 1.0. Use :meth:`~langchain_core.utils.function_calling.convert_to_openai_function()` instead.
  functions = [format_tool_to_openai_function(t) for t in tools]


In [6]:
def call_model(state):
    messages = state['messages']
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

In [7]:
def should_continue(state):
    messages = state['messages']
    last_message = messages[-1]
    # If there is no function call, then we finish
    if "function_call" not in last_message.additional_kwargs:
        return "end"
    # Otherwise if there is, we continue
    else:
        return "continue"

In [8]:
def call_tool(state):
    messages = state['messages']
    # Based on the continue condition
    # we know the last message involves a function call
    last_message = messages[-1]
    # We construct an ToolInvocation from the function_call
    action = ToolInvocation(
        tool=last_message.additional_kwargs["function_call"]["name"],
        tool_input=json.loads(last_message.additional_kwargs["function_call"]["arguments"]),
    )
    # We call the tool_executor and get back a response
    response = tool_executor.invoke(action)
    # We use the response to create a FunctionMessage
    function_message = FunctionMessage(content=str(response), name=action.tool)
    # We return a list, because this will get added to the existing list
    return {"messages": [function_message]}

In [14]:
if __name__ == '__main__':
    # We create a new instance of the workflow
    workflow = StateGraph(AgentState)
    # We add the nodes to the workflow
    workflow.add_node("call_model", call_model)
    workflow.add_node("should_continue", should_continue)
    workflow.add_node("call_tool", call_tool)
    # We add the edges to the workflow
    workflow.add_edge(START, "call_model")
    workflow.add_conditional_edges("call_model", should_continue,
                                   {
                                        "continue": "call_model",
                                        "end": END,
                                   })
    workflow.add_edge("call_model", "call_tool")
    
    # We run the workflow
    runnable_workflow = workflow.compile()
    initial_state = {"messages": [HumanMessage(content="What is 2 times 3?")]}
    final_state = runnable_workflow.invoke(initial_state)
    # We print the final state
    print(final_state)

NameError: name 'ToolInvocation' is not defined